# Py08 - Stream Workers

## Overview

Macrometa GDN allows you to integrate streaming data and take appropriate actions. Most stream processing use cases involve collecting, analyzing, and integrating or acting on data generated during business activities by various sources.

| Stage | Description | 
|  :----:  |    :----:   |
| Collect | Receive or capture data from various data sources.| 
| Analyze | Analyze data to identify interesting patterns and extract information.| 
| Act | Take actions based on the findings. For example, running simple code, calling an external service, or triggering a complex integration.| 
| Integrate | Provide processed data for consumer consumption.| 

You can process streams to perform the following actions with your data:

- Transform data from one format to another. For example, from XML to JSON.
- Enrich data received from a specific source by combining it with databases and services.
- Correlate data by joining multiple streams to create an aggregate stream.
- Clean data by filtering it and by modifying the content in messages. For example, obfuscating sensitive information.
- Derive insights by identifying event patterns in data streams.
- Summarize data with time windows and incremental aggregations.
- Real-time ETL for collections, tailing files, and scraping HTTP endpoints.
- Integrating stream data and trigger actions based on the data. This can be a single service request or a complex enterprise integration flow.

In this tutorial we will build a simple stream worker for finding various heart rate measures like average bpm, minimum bpm etc.

## Pre-requisite

Let's assume your 

- tenant name is an email address
- user password is xxxxx.

if you need to install pyc8, you can run the cell below, otherwise you may skip it.

In [ ]:
!pip install pyC8

## 1. Importing Libraries & Define Variables

The first step is to import the libraries we need and define the variables we will be using in this tutorial. This is also the right place to add your GDN login credentials. i.e. your email and password. You will also need to make sure you have specified the correct federation URL. In this example it is "gdn.paas.macrometa.io" and we are using the default geo fabric "_system".

In [ ]:
import json
from c8 import C8Client

# Variables - Queries
fed_url = "gdn.paas.macrometa.io";
email = "email"; # <-- Email goes here
password = "password"; # <-- password goes here
geo_fabric = "_system";
heart_rates_collection = "HeartRates";
heart_rates_statistics_collection = "HeartRateStatistics";

heart_rate_statistics_worker = "HeartRateStatisticsWorker";
mock_heart_rate_data_generator = "MockHeartRateDataGenerator";

## 2. Connecting to GDN

Now that we have imported the required libraries and added our login details, we can connect to GDN. Do this by running the cell bellow.

You will see the cell output reflect a successful connection. If not go back to the first step and check the details you entered.

In [ ]:
# Initialize the C8 Data Fabric client.
print("\n ------- CONNECTION SETUP  ------")
print("tenant: {}, geofabric:{}".format(email, geo_fabric))
client = C8Client(protocol='https', host=fed_url, port=443,
                email=email, password=password,
                geofabric=geo_fabric)  

# For the "mytenant" tenant, connect to "_system" fabric as tenant admin.
# This returns an API wrapper for the "_system" fabric on tenant 'mytenant'
# Note that the 'mytenant' tenant should already exist.

tenant = client.tenant(email=email, password=password)

sys_fabric = tenant.useFabric('_system')

## 3. Creating Collections

For this Tutorial we require the creation of two collections. The first will be a collection for the heart rates, which will hold our input data. The second will be a collection for the stream worker output which will hold the statistics.

Run the next cell to create these two collections. Note the If-Else statements that are included in this cell. This lets us check to see if the collections exist already in the federation you are using, if they dont exist we will create them.

In [ ]:
# Create a new collection if it does not exist
print("\n 3. CREATE_COLLECTION");
if client.has_collection(heart_rates_collection):
    print("Collection {} exists".format(heart_rates_collection))
else:
    client.create_collection(name=heart_rates_collection)
    print("Collection {} Created!".format(heart_rates_collection))
    
if client.has_collection(heart_rates_statistics_collection):
    print("Collection {} exists".format(heart_rates_statistics_collection))
else:
    client.create_collection(name=heart_rates_statistics_collection)
    print("Collection {} Created!".format(heart_rates_statistics_collection))

**Note: for this to work you need to enable the collection stream from the Macrometa Dashboard. Do this by opening up the dashboard, select collections, find the collection named "HeartRates" and select \"Enable Stream\"**

![disabledstream](./../images/disabled_stream.png)

![enabledstream](./../images/enabled_stream.png)


## 4. Validate Stream Application

Now that we have created our collections we need to decide how we want to simulate the heart rate information. in this section we offer two approaches for simulating this data. 

Option 1, is to use a 3rd party tool called "Mockaroo" that will let you generate the data stream as an external source to Macrometa GDN. This would be closest to a "real world" use case showing data coming into GDN (and our streamworker) from an external source.

Option 2. demostrates how Macrometa GDN Stream Workers can also be used to provide simulation data, or indeed the streaming of data from a collection.

The end result will be the same, however for simplicity please select one option or the other, not both.

Select the option you are going to use, and run the code in the corresponding cell.

### 4.1 Validating heart rate simulator definition

#### Option1: Use mockaroo api to generate the mock heart rate data 

NOTE: If you are using mockaroo API then you need to signup to mockaroo and paste the API Key. You can find API key here https://www.mockaroo.com/myaccount

In this Stream Worker we have created a trigger that triggers every 10 seconds and connects to the Mockaroo API Service and passes the current time stamp to trigger the Mockaroo data generation. 

Then the Stream Worker consumes data received from the external Mockaroo service, selects the persons name and their heart rate and puts it into the collection "HeartRates".

Stream Workers are written in JavaScript, the Cell bellow has two variables: one for yor Mackroo API Key, and the other "dataGeneratorAppDefinition", both are strings that are passed the Key and StreamWorker code, respectively.

In [ ]:
mockarooAPIKey = "XXXX";
dataGeneratorAppDefinition = '''
@App:name('MockHeartRateDataGenerator')
@App:description('Mock data generator by calling mockaroo api for heart rate')
@App:qlVersion('2')


CREATE TRIGGER HeartRateDataGeneratorTrigger WITH ( interval = 10 sec );

CREATE TABLE HeartRates (name string, bpm int);

CREATE SINK MockarooServiceCallSink WITH (type='http-call', sink.id='mockaroo-service', publisher.url='https://api.mockaroo.com/api/a6e130b0?count=10&key={}', map.type='json', method='GET') (triggered_time string);

CREATE SOURCE MockarooServiceResponseSink WITH (type='http-call-response', sink.id='mockaroo-service', map.type='json', http.status.code='200') (name string, bpm int);


INSERT INTO MockarooServiceCallSink
SELECT time:currentTimestamp() as triggered_time 
FROM HeartRateDataGeneratorTrigger;

-- Note: Consume data received from the external service
@info(name = 'ConsumeProcessedData')
INSERT INTO HeartRates
SELECT name, bpm
FROM MockarooServiceResponseSink;
'''.format(mockarooAPIKey);

#### Option2: Use custom stream worker to generate the heart rate data 

In this Stream Worker we have created a trigger that triggers every 10 seconds and creates random data from a short list of people and possible heart rates and puts it into the collection "HeartRates".

Stream Workers are written in JavaScript, the Cell bellow has one variables, "dataGeneratorAppDefinition", it is a string type variable that is passed StreamWorker code.

In [1]:
dataGeneratorAppDefinition = '''
@App:name("MockHeartRateDataGenerator")
@App:qlVersion("2")

CREATE TRIGGER HeartRateDataGeneratorTrigger WITH ( interval = 10 sec );

CREATE TABLE HeartRates (name string, bpm int);


-- Note: Generating random bpm and name 
@info(name = 'ConsumeProcessedData')
INSERT INTO HeartRates
SELECT 
js:eval("['Vasili', 'Rivalee', 'Betty', 'Jennifer', 'Alane', 'Sarena', 'Bruno', 'Carolee', 'Emmott', 'Andre'][Math.floor(Math.random() * 10)]","string") as name,
js:eval("Math.floor(Math.random() * 40) + 40","int") as bpm
FROM HeartRateDataGeneratorTrigger;
'''

### 4.2 Validating stream worker

Now that we have selected and run the cell code for our chosen method of generating data, we can move on to the next stream worker, and then validate them for correctness, before we publish them, and make them active.

This next cell introduces the second stream worker that reads the heart rates from the collection "HeartRates" in a sliding window of 1 min, and then calculates the min, max and averages for that window, and inserts them into the collection "HeartRateStatistics". it is defined as a string and assigned to the "statisticsAppDefinition" variable.

We then validate the streams by using the "validate_stream_app" method, as shown below.


In [ ]:
statisticAppDefinition = '''
@App:name('HeartRateStatisticsWorker')
@App:description("Calculate the statistics for heart rates")
@App:qlVersion("2")

CREATE SOURCE HeartRates WITH (type = 'database', collection = "HeartRates", collection.type="doc", replication.type="global", map.type='json') (name string, bpm int);

CREATE TABLE HeartRateStatistics (eventTime long, name string, minBpm int, maxBpm int, avgBpm double);


INSERT INTO HeartRateStatistics
SELECT 
    eventTimestamp() as eventTime,
    name as name,
    min(bpm) as minBpm,
    max(bpm) as maxBpm,
    avg(bpm) as avgBpm
FROM HeartRates window sliding_time(1 min)
group by name
''';


print("\n 4. VALIDATE_STREAM_WORKERS ... region {}".format(fed_url));

print("--- Validating Stream Application Definition");
result = client.validate_stream_app(data=statisticAppDefinition);

print("--- Validated HeartRateStatisticsWorker Stream Application Definition {} ---".format(result));
result = client.validate_stream_app(data = dataGeneratorAppDefinition);

print("--- Validated MockHeartRateDataGenerator Stream Application Definition {} ---".format(result));

## 5. Save Stream Application

Upon a successful validation we can save our streamworkers by running the following cell.

In [ ]:
print("\n 5. CREATE_STREAM_WORKERS ... region {}".format(fed_url));
print("--- Creating Stream Application");
#The stream app will be created by default in the local region. Optionally, you can send dclist to deploy stream
result = client.create_stream_app(data=statisticAppDefinition);

print("--- Created Stream Application {} ---".format(heart_rate_statistics_worker));
result = client.create_stream_app(data=dataGeneratorAppDefinition);

print("--- Created Stream Application {} ---".format(mock_heart_rate_data_generator));

## 6. Publish Stream Application

A saved stream worker does not mean it is active. For the Stream Worker to be active it needs to be Published. Run the code in the next cell to publish the stream worker and make it active.

In [ ]:
print("\n6. Activating Stream Workers");
client.activate_stream_app(heart_rate_statistics_worker, True);
client.activate_stream_app(mock_heart_rate_data_generator, True);
print("6. Activated Stream Workers");

## 7. Checking HeartRates documents using C8QL

Now we have activated our stream workers the application is live. However, because we are using a 1 min window, we need to wait for 1 minute before we check for records being created.

A good way to do this is that rather counting 60 seconds, is to load the GDN dashboard, select the "HeartRate" Collection, then "stream" and monitor the console. as soon as their are changes, you will see them reflected here.

![monitoring_heart_rate](./../images/monitoring-HR-stream.png)

Alternatively, wait 60 seconds, and run the code in the cell below.

In [ ]:
# Please wait for 1 minute after executing above step as we used sliding window of 1 minute
cursor = client.execute_query('FOR doc IN HeartRates LIMIT 0, 5 RETURN doc')

docs = [document for document in cursor]

print(json.dumps(docs, indent=4))

## 8. Checking HeartRateStatistics documents using C8QL

Now that we have started to collect data in the HeartRates collection, we want to see our analysis. run the cell below to see the output from our stream worker!

In [ ]:
cursor = client.execute_query('FOR doc IN HeartRateStatistics LIMIT 0, 5 RETURN doc')

docs = [document for document in cursor]

print(json.dumps(docs, indent=4))

## 9.  Delete StreamApp and Collections

Almost done. Lets clean up after ourselves and remove the tutorial from GDN.

In [ ]:
print("\n9. DELETE_DATA");
client.delete_stream_app(heart_rate_statistics_worker);
client.delete_stream_app(mock_heart_rate_data_generator);
client.delete_collection(heart_rates_collection);
client.delete_collection(heart_rates_statistics_collection);
print("StreamApp and Collection deleted");

## Section Completed!

Congratulations! you have completed this tutorial.